In [655]:
import re
import os
import glob
import json
from keras.preprocessing import sequence
import numpy as np
import keras
#from keras.utils.np_utils import probas_to_classes

In [656]:
%config IPCompleter.greedy=True

In [657]:
data = os.path.join('..', 'data-scrapes')
print(data)

..\data-scrapes


In [658]:
import datetime, time
cas = time.time()
stamp = datetime.datetime.fromtimestamp(cas).strftime('%d-%m-%Y-%H-%M-%S')
fileName = ""
fileOut = os.path.join('..', 'dataChanged/' + fileName + '.txt')

In [659]:
fastaFiles = glob.glob(data + "/*.fasta")
annoFiles = glob.glob(data + "/*.txt")
print(annoFiles)

['..\\data-scrapes\\all-human-0001-annotations.txt']


In [660]:
def sortuj(_proteinID, _seq, _fileName):
    fileOut = os.path.join('..', 'dataChanged/' + _fileName + '.txt')
    with open(fileOut, "a") as f:
        f.write(_proteinID +", " + _seq + '\n')

In [661]:
for soubor in fastaFiles:
    print("Converting: %s " % soubor)
    counter = 0
    seq = ''
    proteinID = ''

    with open(soubor, "r") as f:        
        for line in f:
            if line.startswith(">"):                
                counter += 1
                proteinID = line.split("|")[1]
                if seq != '':
                    #print(proteinID)
                    sortuj(proteinID, seq, "proteinIDandSeq")
                    seq = ''
                if counter > 10: break
            else:
                seq += line.strip()
            
        if seq != '':
                    print(proteinID)
                    sortuj(proteinID, seq, "proteinIDandSeq")
                    seq = ''
            

Converting: ..\data-scrapes\all-human-0001.fasta 


In [662]:
fileName = 'selectedProteins'
fileOut = os.path.join('..', 'dataChanged/' + fileName + '.txt')
for soubor in annoFiles:
    print("Converting: %s " % soubor)
    counter = 0
    function = ''
    proteinID = ''
    gpcActivity = '0005524'
    selectedProteins = []

    with open(soubor, "r") as f:        
        for line in f:
            
            proteinID = line.split(" ")[0]
            function = line.split(":")[1][:-3]
            if function == gpcActivity:
                selectedProteins.append(proteinID)
    
    with open(fileOut, "w") as f:
        json.dump(selectedProteins, f)
            

Converting: ..\data-scrapes\all-human-0001-annotations.txt 


In [663]:
seqFile = os.path.join('..', 'dataChanged', 'proteinIDandSeq.txt')
selectedProtFile = os.path.join('..', 'dataChanged', 'selectedProteins.txt')

In [664]:
with open(selectedProtFile, "r") as f:
    selectedProteins = json.load(f)

In [665]:
# moc jich delsich neni, delsi zahodim
maxSeqLen = 500

In [666]:
seq = []
label = []


In [667]:
# rozdeleni na dve tridy, ma GPC aktivitu = 1, a nema aktivitu = 0
with open(seqFile, "r") as f:
    for line in f:
        splitted = line.split(",")
        proteinID = splitted[0].strip()
        sekvence = splitted[1].strip()
        if len(sekvence) > maxSeqLen:
            continue
        seq.append(sekvence)
        if proteinID in selectedProteins:
            label.append(1)
        else:
            label.append(0)

In [668]:
print(label)

[1, 0, 1, 1, 0, 0]


In [669]:
# zamena pismen za vektor cisel
def pismena2cisla(_sekvence):
    vektor = []
    alphabet = ['_', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
    
    try:
        for letter in _sekvence:
            index = alphabet.index(letter)
            vektor.append(index)
        return vektor
    except:
        print("error: something is wrong")
    

In [670]:
pismena2cisla('OLE')

error: something is wrong


In [671]:
seqVektor = []
for i in range(len(seq)):
    vektor = pismena2cisla(seq[i])
    seqVektor.append(vektor)
type(seqVektor[0][0])

int

In [672]:
seqVektor = np.array(seqVektor)
label = np.array(label)
print(type(seqVektor[0][1]))
print(seqVektor)

<class 'int'>
[list([11, 1, 1, 1, 1, 1, 14, 6, 6, 6, 6, 6, 4, 13, 15, 15, 17, 4, 6, 19, 6, 13, 6, 19, 13, 6, 4, 19, 4, 11, 19, 9, 6, 14, 13, 5, 3, 19, 6, 13, 15, 22, 17, 14, 10, 14, 22, 8, 6, 4, 6, 1, 22, 6, 11, 19, 16, 16, 1, 22, 3, 7, 19, 15, 9, 17, 15, 19, 1, 8, 9, 9, 8, 16, 13, 5, 4, 7, 14, 17, 22, 2, 14, 15, 17, 10, 15, 4, 8, 14, 8, 10, 10, 15, 5, 15, 7, 4, 12, 19, 8, 6, 8, 15, 3, 8, 10, 15, 1, 16, 17, 10, 4, 1, 11, 15, 3, 19, 22, 8, 19, 14, 3, 10, 11, 4, 17, 3, 10, 22, 9, 10, 10, 9, 16, 14, 14, 10, 16, 12, 3, 7, 8, 2, 22, 5, 10, 22, 14, 8, 10, 15, 6, 10, 9, 22, 8, 7, 16, 1, 12, 19, 10, 7, 15, 3, 10, 9, 13, 16, 12, 10, 10, 8, 12, 17, 17, 2, 3, 10, 9, 8, 2, 3, 5, 6, 10, 1, 15, 8, 1, 3, 13, 4, 7, 3, 7, 17, 6, 5, 10, 17, 4, 22, 19, 1, 17, 15, 20, 22, 15, 1, 13, 4, 8, 11, 10, 12, 16, 9, 6, 22, 17, 9, 16, 8, 3, 8, 20, 16, 19, 6, 2, 8, 10, 1, 4, 11, 10, 16, 12, 15, 13, 8, 5, 13, 6, 9, 7, 22, 10, 3, 14, 10, 12, 7, 8, 10, 6, 8, 10, 6, 16, 13, 16, 14, 4, 3, 10, 12, 2, 8, 8, 12, 11, 9, 1, 1

In [673]:
# srovnani na stejnou delku, momentalne jsou sekvence(vektory), ruzne dlouhe
seqVektor = sequence.pad_sequences(seqVektor, maxlen = maxSeqLen)
seqVektor[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, 11,  1,  1,  1,  1,  1, 14,  6,  6,  6,  6,  6,  4, 13, 15,
       15, 17,  4,  6, 19,  6, 13,  6, 19, 13,  6,  4, 19,  4, 11, 19,  9,
        6, 14, 13,  5,  3, 19,  6, 13, 15, 22, 17, 14, 10, 14, 22,  8,  6,
        4,  6,  1, 22,  6, 11, 19, 16, 16,  1, 22,  3,  7, 19, 15,  9, 17,
       15, 19,  1,  8,  9,  9,  8, 16, 13,  5,  4,  7, 14, 17, 22,  2, 14,
       15, 17, 10, 15,  4,  8, 14,  8, 10, 10, 15,  5, 15,  7,  4, 12, 19,
        8,  6,  8, 15,  3

In [674]:
print(seqVektor.shape)
print(label.shape)

(6, 500)
(6,)


In [675]:
# rozdeleni dat na trenovaci a testovaci 7:3, asi by se mela ta data jeste randomizovat
seqVektorTrain = seqVektor[:int(len(label)*2/3)]
labelTrain = label[:int(len(label)*2/3)]
seqVektorTest = seqVektor[int(len(label)*2/3):]
labelTest = label[int(len(label)*2/3):]

In [676]:
print(seqVektorTrain.shape)

(4, 500)


In [677]:
from keras.layers import Embedding, Input, Dropout, Flatten, Dense, Activation
from keras.models import Model, Sequential
from keras.optimizers import SGD

In [721]:
numAK = 23
dims = 6
batchSize = 2

In [722]:
# pridavani layeru do modelu: https://keras.io/layers/embeddings/
# https://towardsdatascience.com/deep-learning-4-embedding-layers-f9a02d55ac12
# kazdy add prida dalsi layer
model = Sequential()
model.add(Embedding(numAK, dims, input_length = maxSeqLen))
model.add(Flatten())
model.add(Dense(25, activation = 'sigmoid'))
model.add(Dense(1, activation = 'sigmoid'))

In [723]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 500, 6)            138       
_________________________________________________________________
flatten_21 (Flatten)         (None, 3000)              0         
_________________________________________________________________
dense_44 (Dense)             (None, 25)                75025     
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 26        
Total params: 75,189
Trainable params: 75,189
Non-trainable params: 0
_________________________________________________________________


In [724]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(),
metrics=['accuracy'])

In [725]:
hist = model.fit(seqVektorTrain, labelTrain,
                  batch_size = batchSize,
                  epochs = 6,                  
                  verbose=1,
                    validation_data = (seqVektorTest, labelTest))   

Train on 4 samples, validate on 2 samples
Epoch 1/6
4/4 [==============================] - 10s 2s/step - loss: 0.5796 - acc: 0.7500 - val_loss: 1.2139 - val_acc: 0.0000e+00
Epoch 2/6
4/4 [==============================] - 0s 18ms/step - loss: 0.5784 - acc: 0.7500 - val_loss: 1.2203 - val_acc: 0.0000e+00
Epoch 3/6
4/4 [==============================] - 0s 6ms/step - loss: 0.5773 - acc: 0.7500 - val_loss: 1.2261 - val_acc: 0.0000e+00
Epoch 4/6
4/4 [==============================] - 0s 13ms/step - loss: 0.5762 - acc: 0.7500 - val_loss: 1.2322 - val_acc: 0.0000e+00
Epoch 5/6
4/4 [==============================] - 0s 8ms/step - loss: 0.5752 - acc: 0.7500 - val_loss: 1.2377 - val_acc: 0.0000e+00
Epoch 6/6
4/4 [==============================] - 0s 1ms/step - loss: 0.5742 - acc: 0.7500 - val_loss: 1.2431 - val_acc: 0.0000e+00


In [730]:
hist.model.evaluate(seqVekthist.model.metrics_namesorTrain, labelTrain)

4/4 [==============================] - 0s 3ms/step


[0.5708752870559692, 0.75]

In [731]:
hist.model.metrics_names

['loss', 'acc']

In [732]:
#hist.model.get_weights()

In [736]:
print(label[0])
q = model.predict( np.array( [seqVektor[0],] ))
q

1


array([[0.70680565]], dtype=float32)

In [734]:
model.predict_classes(seqVektor)

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1]])

# tak to neni prilis dobra predikce 

In [737]:
# functional api je pry lepsi

In [738]:
input = Input(shape=(maxSeqLen,))

In [739]:
embedding = Embedding(23, dims)(input)

In [740]:
x = Flatten()(embedding)
x = Dense(25, activation='sigmoid')(x)
x = Dense(1)(x)

In [741]:
output = Activation('sigmoid')(x)

In [742]:
model2 = Model([input], output)
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 500)               0         
_________________________________________________________________
embedding_26 (Embedding)     (None, 500, 6)            138       
_________________________________________________________________
flatten_22 (Flatten)         (None, 3000)              0         
_________________________________________________________________
dense_46 (Dense)             (None, 25)                75025     
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 26        
_________________________________________________________________
activation_8 (Activation)    (None, 1)                 0         
Total params: 75,189
Trainable params: 75,189
Non-trainable params: 0
_________________________________________________________________


In [743]:
model2.compile(loss='binary_crossentropy',
              optimizer=SGD(),
metrics=['accuracy'])

In [749]:
hist2 = model2.fit(seqVektorTrain, labelTrain,
                  batch_size = batchSize,
                  epochs = 2,                  
                  verbose=1,
                    validation_data = (seqVektorTest, labelTest))  

Train on 4 samples, validate on 2 samples
Epoch 1/2
4/4 [==============================] - 0s 68ms/step - loss: 0.8573 - acc: 0.2500 - val_loss: 0.4753 - val_acc: 1.0000
Epoch 2/2
4/4 [==============================] - 0s 6ms/step - loss: 0.8358 - acc: 0.2500 - val_loss: 0.4964 - val_acc: 1.0000


In [750]:
#hist.history
print(label)
#model2.predict(seqVektor)
#model2.predict( np.array( [seqVektor[0],] ))
prob = model2.predict(seqVektor)
#y_classes = keras.utils.np_utils.probas_to_classes(y_proba)
predLabels = prob.argmax(axis=-1)
predLabels

[1 0 1 1 0 0]


array([0, 0, 0, 0, 0, 0], dtype=int64)

In [752]:
hist2.model.predict(seqVektor)

array([[0.3963757 ],
       [0.38797292],
       [0.39430186],
       [0.39606562],
       [0.392413  ],
       [0.3901479 ]], dtype=float32)

In [758]:
len(hist2.model.output_layers)

1